<a href="https://colab.research.google.com/github/arthurziegler/pytorch-deep-learning-course/blob/main/Notebooks/PyTorch_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#!pip install torchtext==0.8.1

In [19]:
import torch
import torch.nn as nn
#import torchtext.data as ttd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [20]:
# Let's make some fake data!
data = {
    "label": [0, 1, 1],
    "data": [
        "I like eggs and ham.",
        "Eggs I like!",
        "Ham and eggs or just ham?",
    ]
}

In [21]:
df = pd.DataFrame(data)

In [22]:
df.head()

,label,data
0,0,I like eggs and ham.
1,1,Eggs I like!
2,1,Ham and eggs or just ham?


In [23]:
#df.to_csv('thedata.csv', index=False)

In [24]:
#!head thedata.csv

In [25]:
def inputPreProcessing(input_data):
    processed_data = pd.DataFrame()

    # Add space before final ponctuation to prepare for split
    for index, row in input_data.iteritems():
        if re.match(r'\W', row[-1]):
            input_data.iloc[index] = row[:-1] + ' ' + row[-1]

    processed_data = input_data.str.lower()
    processed_data = processed_data.str.rsplit(expand=True)

    # Change padding 'None' from post to pre-padding
    processed_data = processed_data.to_numpy()
    for index, document in enumerate(processed_data):
        while(document[-1] == None):
            document = np.roll(document, 1)
        processed_data[index] = document

    # Create dictionary that will work as the vocabulary of the dataset
    dataset = list(processed_data.flatten())
    current_idx = 2 #leave 0 and 1 for padding and unknow words
    word2idx = {}
    word2idx[None] = 1
    word2idx['<Unknown>'] = 0
    for word in dataset:
        if word not in word2idx:
            word2idx[word] = current_idx
            current_idx += 1
    print(word2idx)
    print(processed_data)

    # Change tokens from word to integer
    for idx_doc, document in enumerate(processed_data):
        for idx_word, word in enumerate(document):
            processed_data[idx_doc, idx_word] = word2idx[word]
            #print(idx_doc, document, idx_word, word)
    
    print(processed_data)
    return processed_data


input_processed = inputPreProcessing(df['data'])
input_processed

{None: 1, '<Unknown>': 0, 'i': 2, 'like': 3, 'eggs': 4, 'and': 5, 'ham': 6, '.': 7, '!': 8, 'or': 9, 'just': 10, '?': 11}
[[None 'i' 'like' 'eggs' 'and' 'ham' '.']
 [None None None 'eggs' 'i' 'like' '!']
 ['ham' 'and' 'eggs' 'or' 'just' 'ham' '?']]
[[1 2 3 4 5 6 7]
 [1 1 1 4 2 3 8]
 [6 5 4 9 10 6 11]]


C:\Users\arthu\anaconda3\envs\Deep Learning\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


array([[1, 2, 3, 4, 5, 6, 7],
       [1, 1, 1, 4, 2, 3, 8],
       [6, 5, 4, 9, 10, 6, 11]], dtype=object)

In [26]:
#ex = dataset.examples[0]

In [27]:
#type(ex)

In [28]:
#ex.data

In [29]:
#ex.label

In [30]:
#train_dataset, test_dataset = dataset.split(0.66) # default is 0.7

In [31]:
#for ex in train_dataset.examples:
#  print(ex.data)

In [32]:
#TEXT.build_vocab(train_dataset,)

In [33]:
#vocab = TEXT.vocab
#type(vocab)

In [34]:
#vocab.stoi

In [35]:
#vocab.itos

In [51]:
from sklearn.model_selection import train_test_split
tr_inputs, tst_inputs, tr_labels, tst_labels = train_test_split(input_processed, df['label'].to_numpy(), test_size=0.33)

In [52]:
tr_inputs = torch.tensor(tr_inputs.astype(np.int32))
tst_inputs = torch.tensor(tst_inputs.astype(np.int32))
tr_labels = torch.tensor(tr_labels.astype(np.int32))
tst_labels = torch.tensor(tst_labels.astype(np.int32))

In [55]:
tr_labels

tensor([0, 1], dtype=torch.int32)

In [54]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [57]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
bs = 2

train_data = TensorDataset(tr_inputs, tr_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

test_data = TensorDataset(tst_inputs, tst_labels)
test_sampler = SequentialSampler(tst_inputs)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

In [ ]:
#train_iter, test_iter = ttd.Iterator.splits(
#        (train_dataset, test_dataset), sort_key=lambda x: len(x.data),
#       batch_sizes=(2, 2), device=device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
for inputs, targets in train_iter:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


inputs: tensor([[ 1,  1,  1,  2,  7,  9,  4],
        [ 3,  6,  2, 10,  8,  3,  5]], device='cuda:0') shape: torch.Size([2, 7])
targets: tensor([1, 1], device='cuda:0') shape: torch.Size([2])


In [58]:
for inputs, targets in train_dataloader:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

inputs: tensor([[ 1,  2,  3,  4,  5,  6,  7],
        [ 6,  5,  4,  9, 10,  6, 11]], dtype=torch.int32) shape: torch.Size([2, 7])
targets: tensor([0, 1], dtype=torch.int32) shape: torch.Size([2])


In [ ]:
for inputs, targets in test_iter:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

inputs: tensor([[7, 9, 2, 6, 3, 0]], device='cuda:0') shape: torch.Size([1, 6])
targets: tensor([0], device='cuda:0') shape: torch.Size([1])


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [59]:
for inputs, targets in test_dataloader:
  print("inputs:", inputs, "shape:", inputs.shape)
  print("targets:", targets, "shape:", targets.shape)
  break

inputs: tensor([[1, 1, 1, 4, 2, 3, 8]], dtype=torch.int32) shape: torch.Size([1, 7])
targets: tensor([1], dtype=torch.int32) shape: torch.Size([1])


In [ ]:
# Exericise: Figure out which sequence of integers goes with which sentence.